In [ ]:
import os
os.chdir('dummy_to_replace_with_blood_aging_dir_path/code')
# print(os.getcwd())


import sys
import shutil
# caution: path[0] is reserved for script path (or '' in REPL)

import importlib
import itertools
import metacells as mc
import anndata as ad
import pandas as pd
import numpy as np
import scipy.cluster.hierarchy
import scipy.spatial.distance
import matplotlib
import seaborn as sb
import matplotlib.pyplot as plt
import pathlib
import logging
import re
import collections
import sklearn
import pickle

from generic import generic_utils
from mds import mds_analysis_params
from mds import mds_analysis
from generic import mc_utils
from mds import arch_mutation_interface_and_utils
from mds import karyotype_utils
from mds import clinical_data_interface_and_utils
from sc_rna_seq_preprocessing import sc_rna_seq_preprocessing_params

plt.rcParams["patch.force_edgecolor"] = False
plt.rcParams['patch.linewidth'] = 0
plt.rcParams['patch.edgecolor'] = 'none'
# plt.rcParams['scatter.edgecolors'] = 'black' # didnt affect sb.scatterplot

%matplotlib widget
%load_ext autoreload
%autoreload 2

def get_mds_params():
    return mds_analysis_params.MDS_ANALYSIS_PARAMS


def get_sc_rna_seq_preprocessing_params():
    return sc_rna_seq_preprocessing_params.SC_RNA_SEQ_PREPROCESSING_PARAMS

plt.subplots()
plt.close('all')

In [ ]:
for dataset_name in [
    'illu_mds',
    'ult_mds',
]:
    for mc_model_name in [
        'final_mds_cyto_normal_excluding_atlas',
        *(['final_normal_pb_atlas'] if dataset_name == 'illu_mds' else []),
    ]:
        mc_model_paths = mds_analysis_params.get_mc_model_paths(mc_model_name, dataset_name=dataset_name)
        if mc_model_name == 'final_normal_pb_atlas':
            mc_ad_file_path = get_mds_params()['nimrod_oren_atlas']['mc_ad_file_path']
            c_ad_file_path = get_mds_params()['nimrod_oren_atlas']['c_ad_file_path']
        else:
            mc_ad_file_path = mc_model_paths['metacells_with_projection_ad']
            c_ad_file_path = mc_model_paths['cells_with_metacell_attrs_ad']
        mc_ad = ad.read_h5ad(mc_ad_file_path)
        c_ad = ad.read_h5ad(c_ad_file_path)

        clinical_df = clinical_data_interface_and_utils.get_minimal_clinical_data_df()
        c_ad.obs = sc_rna_seq_preprocessing_params.get_df_with_numbered_donor_id(c_ad.obs)
        c_ad.obs = sc_rna_seq_preprocessing_params.overwrite_according_to_exp_name_and_donor_id(c_ad.obs)
        c_ad.obs = clinical_data_interface_and_utils.get_df_with_diagnosis_and_diagnosis_class_by_exp_name_and_donor_id_and_clinical_df(
            c_ad.obs, clinical_df)
        sc_rna_seq_preprocessing_params.write_misc_numbered_donor_info_pickle(c_ad, mc_model_paths['misc_numbered_donor_info_pickle'])
        
        feature_data = mds_analysis.get_donor_feature_data(c_ad, mc_ad, dataset_name=dataset_name) # NOTE: this is intentionally after fixing metadata in c_ad.obs

        # pathlib.Path('temp/feature_dfs_etc').mkdir(parents=True, exist_ok=True)
        # feature_data['ext_donor_feature_df'].to_csv(f'temp/feature_dfs_etc/{dataset_name}__{mc_model_name}__ext_donor_feature_df.csv', index=False)
        feature_data['ext_donor_feature_df'].to_csv(mc_model_paths['ext_donor_feature_df'], index=False)
        generic_utils.write_text_file(mc_model_paths['feature_cols'], '\n'.join(sorted(feature_data['feature_cols'])))
                                                    


In [ ]:
unified_mc_model_name = 'final_mds_cyto_normal_excluding_atlas'
# unified_mc_model_name = 'final_normal_pb_atlas'
dataset_name = 'ult_mds'
dataset_name = 'illu_mds'

unified_mc_model_paths = mds_analysis_params.get_mc_model_paths(unified_mc_model_name, dataset_name=dataset_name)
unified_mc_ad_file_path = unified_mc_model_paths['metacells_with_projection_ad']
unified_c_ad_file_path = unified_mc_model_paths['cells_with_metacell_attrs_ad']

print(f'the last modification time of the input files are:')
if unified_c_ad_file_path is not None:
    print(f'unified_c_ad_file_path: {generic_utils.get_file_last_modification_time_str_repr(unified_c_ad_file_path)}')
print(f'unified_mc_ad_file_path: {generic_utils.get_file_last_modification_time_str_repr(unified_mc_ad_file_path)}')
print('please make sure all are as expected.')

info_for_karyo = karyotype_utils.get_info_for_karyo_estimation(unified_mc_model_name, unified_mc_ad_file_path, unified_c_ad_file_path, dataset_name=dataset_name)

In [ ]:
DPI = 100
DPI = 300

STOP_AFTER_MC_MOVING_MEDIAN = False
STOP_AFTER_MC_MOVING_MEDIAN = True

stop_after_writing_cna_hists_and_dfs = False
stop_after_writing_cna_hists_and_dfs = True

CORRECT_BY_GC = False
CORRECT_BY_GC = True

forced_mc_model_names = [
    # 'final_only_N211_only_demux_16_01_22_1',
]
forced_donor_ids = [
    'N200',
    # 'N211',
]

# sorted_gene_names = var_df[(var_df['unified_end_pos_bin_i'] >= 0) & curr_gene_mask].sort_values('unified_end_pos_bin_i').index.to_numpy()

karyotype_utils.estimate_karyo(
    info_for_karyo, forced_mc_model_names=forced_mc_model_names, 
    forced_donor_ids=forced_donor_ids, correct_by_gc=CORRECT_BY_GC, dpi=DPI,
    stop_after_mc_moving_median=STOP_AFTER_MC_MOVING_MEDIAN, allow_donor_also_in_atlas=True,
    dataset_name=dataset_name,
)


In [ ]:
all_cna_attrs_df, donor_agg_cna_df = karyotype_utils.get_all_cna_attrs_df_and_donor_agg_cna_df(info_for_karyo)

all_cna_attrs_df.to_csv(get_mds_params()['all_cna_attrs_df_csv_file_path'], index=False)
donor_agg_cna_df.to_csv(get_mds_params()['donor_agg_cna_df_csv_file_path'], index=False)